# Converting the raw data ready to pocess, to csv file


### all the imports

In [2]:
import os
import pandas as pd


### getting all the files from the directory

In [3]:
def get_files():
    all_file_path = []
    """walk through the data dir to its sub-dirs and gets all the file with .review extension and return all the file_paths list"""
    # looping through the directory and the sub-directory
    for root, dirs, files in os.walk("data"):
        for file in files:
            # getting the file end with .review
            if file.endswith(".review"):
                # joining the file path with the actual full path of the file 
                file_path = os.path.join(root, file)
                # appending the file_path in the list 
                all_file_path.append(file_path)
                
    print(all_file_path)
    return all_file_path


In [4]:
# check
all_file_path = get_files()

['data/electronics/unlabeled.review', 'data/electronics/negative.review', 'data/electronics/positive.review', 'data/books/negative.review', 'data/books/positive.review', 'data/dvd/unlabeled.review', 'data/dvd/negative.review', 'data/dvd/positive.review', 'data/kitchen_&_housewares/unlabeled.review', 'data/kitchen_&_housewares/negative.review', 'data/kitchen_&_housewares/positive.review']


## categorized the files into list


In [5]:
def categorise_files(all_file_path):
    """get all file path and categorised the files into positive, negative and unlabeled review and return the file path list as tuple"""
    
    pos_review_f_path = []
    neg_review_f_path=[]
    unl_review_f_path = []
    
    for file in all_file_path:
        if "positive" in file:
            pos_review_f_path.append(file)
        elif "negative" in file:
            neg_review_f_path.append(file)
        else:
            unl_review_f_path.append(file)
    return pos_review_f_path, neg_review_f_path, unl_review_f_path

In [6]:
#check
categorise_files(all_file_path)

(['data/electronics/positive.review',
  'data/books/positive.review',
  'data/dvd/positive.review',
  'data/kitchen_&_housewares/positive.review'],
 ['data/electronics/negative.review',
  'data/books/negative.review',
  'data/dvd/negative.review',
  'data/kitchen_&_housewares/negative.review'],
 ['data/electronics/unlabeled.review',
  'data/dvd/unlabeled.review',
  'data/kitchen_&_housewares/unlabeled.review'])

## reading the files

In [7]:
def read_file(file_path):
    """reads the file from file path and return all lines in list"""
    with open(file_path) as file:
        items = file.readlines()
    return items

In [8]:
# check
items = read_file(all_file_path[0])

## making the list of review after reading the file

In [9]:
def build_review_list(items):
    """gets all the items, prepare a list of each review and returns the list of reviews"""
    reviews = []
    review = []
    for item in items:
        # remove \n from the item
        item = item[:-1]
        # get the file inside the review block not including the start and the end block
        if item != '<review>' and item != '</review':
            review.append(item)
            
        #  adding the review to the review list  
        if item == '</review>':
            reviews.append(review)
            review = []
    return reviews
   

In [10]:
#check
list_of_reviews = build_review_list(items)

### processing the review and making it ready to transfer into the dataframe

In [11]:
def process_review_list(list_of_reviews):
    """gets the list of reviews, seperate the headings and values and returns as list of dictionary"""
    dict_review_list = []
    previous_key = None
    
    #iterating through reviews
    for review in (list_of_reviews):
        dict_review = {}
        #iterating through a single review
        for num, item in enumerate(review):
            #eliminating null values
            if item != "":
                #getting and heading and assigning with corresponding values
                if item[0] =="<" and item[1] != "/":
                    dict_review[item] = review[num + 1]
                    previous_key = item
                #checking if any values are missing and reassigning them
                if item not in dict_review.values() and item[0] != '<':
                    dict_review[previous_key] += item
        # when a review is ready adding it to the list
        dict_review_list.append(dict_review)
    return dict_review_list

### converting the list of the dict into the datarame to convert into the csv file or process straightly


In [12]:
# creating the data from list of dict using lambda
to_dataframe = lambda list_of_dict: pd.DataFrame(list_of_dict)

### converting the dataframe into the csv file and saving into the directory


In [13]:
# convert and save to csv file
save_to_csv = lambda df, file_name: df.to_csv(file_name, index=False, encoding='utf-8')

### combining all the process to get single list of review of each category file found in directory

In [14]:

def convert_to_csv():
    dir = os.path.abspath(os.getcwd()) + "/data/csv_files/"
    
    file_name = ('positive_review', 'negative_review', 'unlabled_review')

    # get all the files
    all_file_path = get_files()
    
    #categorised the files in the list
    categories = categorise_files(all_file_path)
    
    #read all the files from each category:
    for num, category in enumerate(categories):
        
        # hold all the items of each category in one list 
        items_holder = []
        
        for file_path in category:
            # getting all the items
            items = read_file(file_path)
            items_holder.extend(items)
        
        # build the list of the items in one category
        list_of_reviews = build_review_list(items_holder)
        
        # buid of list of processed review in dictionaries
        list_of_dict_reviews = process_review_list(list_of_reviews)
        
        # build of dataframe from processed_list
        data_frame = to_dataframe(list_of_dict_reviews)
        
        # save the dataframe into the csv file
        save_to_csv(data_frame, file_name=(dir + file_name[num]))
        
            

In [15]:
# convert to csv
convert_to_csv()

['data/electronics/unlabeled.review', 'data/electronics/negative.review', 'data/electronics/positive.review', 'data/books/negative.review', 'data/books/positive.review', 'data/dvd/unlabeled.review', 'data/dvd/negative.review', 'data/dvd/positive.review', 'data/kitchen_&_housewares/unlabeled.review', 'data/kitchen_&_housewares/negative.review', 'data/kitchen_&_housewares/positive.review']


In [16]:
# options for pandas
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
# pd.set_option('display.width', 1000)
# pd.set_option('display.max_colwidth', None)

In [17]:
# opening  csv file
open_review = lambda file: pd.read_csv(file)

## Labeling Unlabled data

In [18]:
df_unlabeled = open_review("./data/csv_files/unlabled_review")

/var/folders/wl/4d9cyrr5325_wrhc_bg_xh1h0000gn/T/ipykernel_33684/1404158848.py:2: DtypeWarning: Columns (11,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  open_review = lambda file: pd.read_csv(file)


In [19]:
df_unlabeled.head()

,<unique_id>,<asin>,<product_name>,<product_type>,<helpful>,<rating>,<title>,<date>,<reviewer>,<reviewer_location>,<review_text>,<br />,"<br />the shimmying, im less than impressed with. the tutorial is less then a minute long, and she only explains knee shimmies. again a few good pointers, but their so basic i already knew.","<br />the samba tutorial is actually ok, her brazillian squares are odd and overexagerated, not well executed, and more of the silly not in sync audio overdubbing. the only things she performs well are the basic moves anyone thats ever entered a club can do. i like how she does variations and thinks of modifications and steps for beginners that they can build up to. heck, i might even take away a few moves from this, but i have a feeling i'd get a much better instruction from a primarily latin/samba/salsa dancer. funny how the real pro's do it better, eh? don't think shes a pro, because trust me she isn't. well intentioned, but just not a very good dancer im sorry to say.","<br />the salsa tutorial is ok, i think this lady knows the latin dances best. she gives ok visual instructions, pointing out alignment in the torso and adding modifications to steps. you could really go out with a few of these moves to a club i think, but again you'd be better off with a primarily salsa/latin/flamenco teacher. this is probably the best, and suprisingly compitent. more bad audio, but thats the way it goes.","<br />the flamenco tutorial is good, showing pretty good technical form. her flamenco hands are well done (you see where she gets the disaster of the basic egyptian hand undulations"
0,B00006HYUB:power_ups!_everything_you_need_to_k...,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,19 of 24,5.0,Power UPS! Everything you need to know.,"April 11, 2006",404knowmore,California,First of all why do you want a power UPS? Ther...,NaN,NaN,NaN,NaN,NaN
1,B00006HYUB:good_product:s._reiss,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,5 of 8,5.0,Good Product,"January 31, 2006",S. Reiss,"Annandale, NJ",We bought this to backup the iMac G5 that I go...,NaN,NaN,NaN,NaN,NaN
2,B00006HYUB:correction_to_earlier_review:j._zim...,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,17 of 19,5.0,Correction To Earlier Review,"December 12, 2005","J. Zimmerlin ""JimZim""","Grover Beach, California USA",It's unfortunate that an earlier review indica...,NaN,NaN,NaN,NaN,NaN
3,B00006HYUB:nedessary_accessory_for_all_compute...,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,6 of 7,5.0,Nedessary accessory for all computer users,"September 24, 2005","Alan Brody ""Al""","Cincinnati, OH USA",I replaced an older APC uninterruptible power ...,NaN,NaN,NaN,NaN,NaN
4,B00006HYUB:don't_risk_being_without_one!:there...,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,3 of 4,5.0,Don't risk being without one!,"September 16, 2005",Theresa L. Smith,"Blue Springs, MO",I already knew how good this product was befor...,NaN,NaN,NaN,NaN,NaN


In [20]:
# drop the unncecessary cols=
req_columns = df_unlabeled.columns.to_list()[:11]

# update the data_frame 
df_unlabeled = df_unlabeled[req_columns]

In [21]:
df_unlabeled.head()

,<unique_id>,<asin>,<product_name>,<product_type>,<helpful>,<rating>,<title>,<date>,<reviewer>,<reviewer_location>,<review_text>
0,B00006HYUB:power_ups!_everything_you_need_to_k...,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,19 of 24,5.0,Power UPS! Everything you need to know.,"April 11, 2006",404knowmore,California,First of all why do you want a power UPS? Ther...
1,B00006HYUB:good_product:s._reiss,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,5 of 8,5.0,Good Product,"January 31, 2006",S. Reiss,"Annandale, NJ",We bought this to backup the iMac G5 that I go...
2,B00006HYUB:correction_to_earlier_review:j._zim...,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,17 of 19,5.0,Correction To Earlier Review,"December 12, 2005","J. Zimmerlin ""JimZim""","Grover Beach, California USA",It's unfortunate that an earlier review indica...
3,B00006HYUB:nedessary_accessory_for_all_compute...,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,6 of 7,5.0,Nedessary accessory for all computer users,"September 24, 2005","Alan Brody ""Al""","Cincinnati, OH USA",I replaced an older APC uninterruptible power ...
4,B00006HYUB:don't_risk_being_without_one!:there...,B00006HYUB,APC Back-UPS ES 500 Backup Battery and Surge P...,electronics,3 of 4,5.0,Don't risk being without one!,"September 16, 2005",Theresa L. Smith,"Blue Springs, MO",I already knew how good this product was befor...


In [22]:
df_unlabeled[df_unlabeled['<rating>'] == 3]

,<unique_id>,<asin>,<product_name>,<product_type>,<helpful>,<rating>,<title>,<date>,<reviewer>,<reviewer_location>,<review_text>


In [23]:
# there are no data with the ratings 3 and when going through the data, the rating below 3 got negative reveiws 
# and above 3 got the postive review 
# hence we will update the data_frame with new column called outcome and fill with negative and postive outcome

df_unlabeled['<outcome>'] = df_unlabeled['<rating>'].apply(lambda x: 'positive' if x >=3 else 'negative')

In [24]:
# check how many rows of data are negative or postive

print(len(df_unlabeled[df_unlabeled['<outcome>'] == 'negative']))
print(len(df_unlabeled[df_unlabeled['<outcome>'] == 'positive']))

9955
54724


In [25]:
# so there is huge difference in teh postive and the negative reviews, in supervised model we can just take equal amount 
# of the data for best traning of the model so , we  are going to take equal amount of reviews of each category
# 9995
negative_df = df_unlabeled[df_unlabeled['<outcome>'] == 'negative']
positive_df = df_unlabeled[df_unlabeled['<outcome>'] == 'positive']

# now just select 9955 random values
positive_df = positive_df.sample(n=9955, random_state=42)




In [26]:
# check the length of the both data frame
print(len(negative_df))
print(len(positive_df))

9955
9955


## Combining both data

In [27]:
# reading the csv files
df_positive = open_review("./data/csv_files/positive_review")
df_negative = open_review("./data/csv_files/negative_review")


In [28]:
# adding outcome label to the both dataset
df_positive['<outcome>'] = 'positive'
df_negative['<outcome>'] = 'negative'

# # combining the both dataframe and making it as a one
# final_df = pd.concat([df_positive_new, df_negative_new])

In [29]:
# we can replace the nan reviewer to "Anonymous User" to say these person does not want to disclose their names on reviews
# it can be possible that the reviewer doesn't have amazon account, or he posted anonymoulsly or there is an technical issues in the database
df_positive.loc[df_positive['<reviewer>'].isna()].head()

,<unique_id>,<asin>,<product_name>,<product_type>,<helpful>,<rating>,<title>,<date>,<reviewer>,<reviewer_location>,<review_text>,<outcome>
238,"B000EP5MOA:excellent_ipod,_my_favorite_so_far.:",B000EP5MOA,Apple 4 GB iPod Nano Green (2nd Generation): E...,electronics,7 of 7,5.0,"Excellent iPod, my favorite so far.","November 22, 2006",NaN,NaN,I love the new nanos. I don't think I have any...,positive
316,B00001P4ZD:fits_my_fat_head...:,B00001P4ZD,Koss KTX-Pro Portable Stereophone: Electronics,electronics,1 of 2,4.0,Fits my fat head...,"August 21, 2002",NaN,NaN,I stretched them till they fit comfortably on ...,positive
433,B00000J0O2:it_does_the_job:,B00000J0O2,HP Microfine Toner Cartridge for LaserJet 5L/6...,electronics,3 of 3,5.0,It does the job,"June 20, 2000",NaN,NaN,Well the most important thing about buying ink...,positive
438,"B000058E2B:why_i_still_like_this_pda,_continued:",B000058E2B,Compaq iPAQ 3635 Pocket PC Bundle (with Compac...,electronics,NaN,4.0,"Why I still like this PDA, continued","April 4, 2004",NaN,NaN,I found out the way to copy photos from my Com...,positive
532,B00003CWE5:buy_these_cables!:,B00003CWE5,Monster Cable MV2CV-1M Monster Video 2 Compone...,electronics,4 of 12,5.0,Buy these cables!,"March 15, 2001",NaN,NaN,These cables are super with my new DVD player....,positive


In [30]:
# lambda to combine data_frame
combine_df = lambda df_1, df_2: pd.concat([df_1, df_2])

In [31]:
# concating both similar data_frame
df_positive = combine_df(positive_df, df_positive)

df_negative = combine_df(negative_df, df_negative)

In [32]:
# check the length of the data frame
print(len(df_positive))
print(len(df_negative))

13955
13955


In [33]:
# combining the data_frame as one
final_df = combine_df(df_positive, df_negative)
print(len(final_df))

27910


In [34]:
final_df['<rating>'].isnull().sum()

0

# Processing the overall data

In [35]:
# remove the unique id and make asin as a unique id
# remove the unwanted text from product name
# in helpful column make the data in fractions or ratio to represent mathmatically
# convert the date column into the date type
# remove the unwanted text from the review text

In [36]:
# process to follow

# process the positive review file
# process the negative review file
# merge the both file 
# randomised the data
# split the data into train, evaluation and test data
# train the model 
# evaluate the model
# test the model
# dump the model into the file so that we don't have to train the new model time to time


In [37]:
# reviews_text = df_positive['<review_text>']


In [38]:
# not using because of the work_around found on the later cells
# def limit_words_in_reviews(data_frame, column_name, num_words):
#     """
#     This function takes data_frame , column name and limit value then it will limit
#     the words by that numbers and return the pandas dataframe column
#     """
#     filtered_reviews = []
#     for review_text in data_frame[column_name].to_list():
#         words = review_text.split()
#         filtered_reviews.append(" ".join(words[:num_words]))
#     df_col = pd.DataFrame(filtered_reviews, columns=[column_name])
#     return df_col

In [39]:
# # limiting on the positive review file
# df_positive['<review_text>'] = limit_words_in_column(df_positive, "<review_text>", 138)

# # limiting on the negative review file
# df_negative['<review_text>'] = limit_words_in_column(df_negative, "<review_text>", 138)


In [40]:
# not required work around found
# def check_words_counts(data_frame, column_name, num_limit):
#     column = data_frame[column_name]
#     for num, row in enumerate(column):
#         row = row.split(" ")
#         if len(row) > num_limit:
#             print(num +1 , len(row))

In [41]:
 # not required work around found
 
# # validating the positive review file
# check_words_counts(df_positive, '<review_text>', 138)

# # validating the negative review file
# check_words_counts(df_negative, '<review_text>', 138)


In [42]:
# columns_to_include = ['<review_text>', '<rating>', '<title>', '<outcome>']
# # creating processed dataframe from positive review dataframe
# df_positive_new = df_positive[columns_to_include]

# # creating the processed dataframe form negative review dataframe
# df_negative_new = df_negative[columns_to_include]


In [43]:
columns_to_include = ['<review_text>', '<rating>', '<title>', '<outcome>']
final_df = final_df[columns_to_include]

In [44]:
final_df.tail()

,<review_text>,<rating>,<title>,<outcome>
3995,This thermometer lasted about a month before g...,1.0,Bad Product!,negative
3996,"Bought my first Vornado years ago, and is the ...",1.0,Disappointed,negative
3997,More money then its worth. Makes alot of nois...,2.0,Very Average,negative
3998,Mine broke on the 3rd use! No need to say any...,1.0,I love OXO products but this was a huge disapp...,negative
3999,Worked great the first couple of times. I did...,2.0,Don't buy it!,negative


In [45]:
# randomizing the the data inside the dataframe 
final_df = final_df.sample(frac=1, replace=False, random_state=42)

In [46]:
len(final_df)

27910

In [47]:
final_df.head()

,<review_text>,<rating>,<title>,<outcome>
6611,[ This is almost the same as my 628 review ] I...,4.0,Best remote I've used,positive
23997,Call me a Jane Austen Purist but i hated this ...,2.0,"I love Jane Austen, But I HATED it",negative
11643,I was looking foreward to getting this product...,1.0,THIS HARD DRIVE SUCKS!!!!,negative
48300,I've been using the Space Bag for over 2 years...,5.0,Space Bag,positive
55508,I love my Circulon frying pan but would spend ...,5.0,Circulon nylon cleasning brush.,positive


In [48]:
len(final_df)

27910

In [49]:
# transforming the string data into the numerical format to process

# on the dataframe we prepared we have categorial, textual and numerical data so we need to have different
# techniques to convert them into the numerical state which model can understand

# for textual data ---> BoW or TF-IDF can be used to represent into vector format
# for catoegorical data --> one hot encoder can be used 
# numerical data can be used as it is but need to mention in the model it is numerical data
 

In [50]:
# imports required
# from sklearn.feature_extraction.text import CountVectorizer

In [51]:
# count_vectorizer = CountVectorizer()
# bag_of_words = count_vectorizer.fit_transform(final_df['<review_text>'])

In [52]:
# lst = count_vectorizer.get_feature_names_out()
# for l in lst:
#     print(l)
# count_vectorizer.vocabulary_

In [53]:

# add arbritary number of words to stop words
def add_to_stop_words(*args, nlp:'language_model'):
    for item in args:
        nlp.Defaults.stop_words.add(item)

# remove arbritary number of item form the stop words
def remove_from_stop_words(*args, nlp:'language_model'):
    for item in args:
        nlp.Defaults.stop_words.remove(item)

In [54]:
# loading the spacy module with english medium library
import spacy
nlp = spacy.load('en_core_web_md')

In [55]:
# adding unecessay words to stopping words
add_to_stop_words(".", "-", "+", "\\", "/", "_", "'", '"', "?", ";", ":", "!", ",", " ",
                  "I", "(", ")", "--","[", "]", "the", "this", "however", "a", "he", "  ", "...", "we", nlp=nlp)

In [56]:
# remove the items from the stopping words with are necessary
# add items here looking from the removed word cloud list
# remove_from_stop_words()

In [57]:
# tokenizing the text using spacy
def tokenize_texts(df:'dataframe', nlp:'language_framework', column_name):
    tokens_list = []
    for items in df[column_name].to_list():
        tokens_list.append(nlp(items))
    return tokens_list
        
    

In [58]:
# check if the words is in stop words lists:
def check_word_in_stop_list(stop_list, word_list):
    for word in word_list:
        if word in stop_list:
            print(f"{word} is in stop list")
            # remove it from the stop list
            remove_from_stop_words(word, nlp=nlp)

In [59]:
# words that can change the sentiment
negative_words = ["n't", 'not', 'no', 'never', 'nothing', 'nowhere', 'noone', 'none', 'neither', 'nor', 'hardly', 'scarcely', 'barely', 'without', 'least', 'few', 'several', 'less']

positive_words = [
    'amazing',
    'awesome',
    'beautiful',
    'best',
    'brilliant',
    'celebrate',
    'charming',
    'cheerful',
    'congratulations',
    'delight',
    'delightful',
    'enjoy',
    'enthusiastic',
    'excellent',
    'exciting',
    'fabulous',
    'fantastic',
    'flourishing',
    'fun',
    'glamorous',
    'good',
    'gorgeous',
    'great',
    'happy',
    'hilarious',
    'impressive',
    'incredible',
    'inspiring',
    'jubilant',
    'joyful',
    'lovely',
    'marvelous',
    'nice',
    'perfect',
    'pleasurable',
    'positive',
    'promising',
    'remarkable',
    'rewarding',
    'satisfying',
    'splendid',
    'successful',
    'superb',
    'terrific',
    'thrilling',
    'triumphant',
    'vibrant',
    'wonderful',
]

# important words list
important_words = []





In [60]:
# checking removing the important negative words from stop list
check_word_in_stop_list(word_list=negative_words, stop_list=nlp.Defaults.stop_words)

n't is in stop list
not is in stop list
no is in stop list
never is in stop list
nothing is in stop list
nowhere is in stop list
noone is in stop list
none is in stop list
neither is in stop list
nor is in stop list
without is in stop list
least is in stop list
few is in stop list
several is in stop list
less is in stop list


In [61]:
# checking removing the important negative words from stop list
check_word_in_stop_list(word_list=positive_words, stop_list=nlp.Defaults.stop_words)

In [62]:
def clean_review(tokens_list, nlp:'language_framework'):
    """filters the stop word from the reviews and returns the list of stop words and the list of refined reviews"""
    words = []
    refined_list = []
    removed_token_list = []
    for tokens in tokens_list:
        for token in tokens:
            # removing the stop words
            if token.text not in nlp.Defaults.stop_words:
                # lemmitization of the words
                word = token.lemma_
                
                # convert n't to not 
                if token.text == "n't":
                    word = 'not'
                
                words.append(word.lower())
            else:
                removed_token_list.append(token.text.lower())
        refined_list.append(words)
        words = []
    
    return(refined_list,removed_token_list)

In [63]:
# tokenizing (this will take time)
tokens_list = tokenize_texts(final_df, nlp, "<review_text>")

In [157]:
type(tokens_list[0])

spacy.tokens.doc.Doc

In [64]:
# removing the stop words from the tokens and sperating them in seprate list 
refined_list, removed_text_list = clean_review(tokens_list, nlp)

In [65]:
def calc_average_words_in_reviews(item_list):
    sum = 0
    for item in item_list:
        sum = sum + len(item)
    return sum/len(refined_list)

calc_average_words_in_reviews(refined_list)

# to reduce filling up zero while padding the sequence to homogeneous size  
# we will take average word length as max length of words in each review 
# we will go with 70 

max_length=70


In [66]:
def limit_words_in_reviews(review_list, num_words):
    """
    limit all the words to 70 words from all the list of tokens or review and return the list
    """
    filtered_reviews = []
    for review_words in review_list:
        review_words = review_words[:70]
        filtered_reviews.append(review_words)
    return filtered_reviews

In [67]:
# limit word reviews
refined_list = limit_words_in_reviews(refined_list, max_length)

In [68]:
def create_vocab(list_of_text):
    """counts the repetition of the words over all the review texts"""
    vocab = {}
    for item in list_of_text:
        if item in vocab:
            vocab[item] += 1
        else:
            vocab[item] = 1
    return vocab

In [69]:
# all the removed stoppign words
removed_vocab = create_vocab(removed_text_list)

# all the added items vocab
flat_list = [token for review in refined_list for token in review]
added_vocab = create_vocab(flat_list)

In [70]:
def fill_words(review_list):
    r_list = []
    rev = []
    for review in review_list:  
        if len(review) <= 35:
           rev =  review * (70 // len(review))
           r_list.append(rev)
           rev = []
        else:
            r_list.append(review)
        
    return r_list

In [71]:
ref_list = fill_words(refined_list)

In [72]:
len(ref_list)

27910

In [73]:
# word cloud for both of the vocabs
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Sample text to create word cloud
text = "Python is an interpreted high-level programming language for general-purpose programming. \
Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes \
code readability with its notable use of significant whitespace."

# Create WordCloud object
wordcloud = WordCloud(width = 800, height = 800, background_color ='white', 
                min_font_size = 10).generate(" ".join(removed_text_list))

wordcloud1 = WordCloud(width = 800, height = 800, background_color ='white', 
                min_font_size = 10).generate(" ".join(added_vocab))

# Display the WordCloud                    
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

# Show plot
plt.show()

In [74]:
# Display the WordCloud                    
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud1) 
plt.axis("off") 
plt.tight_layout(pad = 0) 

# Show plot
plt.show()

In [75]:
# removing stopping words
# reviews = final_df['<review_text>'].to_list()
# tokens_list = []
# for review in reviews:
#     tokens_list.append(nlp(review))
    

In [76]:
# the traning could be done while taking the entities and removing the entities to detect in 
# which contexts the model performs better

### Transforming data into numerical using the tokenizer from the keras module

In [77]:


# transforming each tables
import numpy as np
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

def tokenize_items(tokenizer, item_seq):
    tokenizer.fit_on_texts(item_seq)
    item_seq = tokenizer.texts_to_sequences(item_seq)
    return item_seq

In [78]:
reviews_list = tokenize_items(tokenizer, refined_list)

In [163]:
print(reviews_list[5])

[    4     1    21    26   905     2  2443    40    46  2278   907   297
   318   785   318   351   344   143   107  1958  1381   286  1241  8948
    27   143 26473  2278    52     1    35     2    35   201   270   491
   107   959  1325   215   388   586   685     5  3139   157  2575    14
   107   119  1044     8     2  1122    21   192   107   393    10    54
   369   192    52   647   192    64   218    38    67     6]


In [79]:
# since the reviews are not in homogeneous size we need to pad the sequence to make it homegeneous 
# we averaged the words count for this step, where it adds zero is the length of teh review are not of max_length=70
from keras.utils.data_utils import pad_sequences
reviews_list = pad_sequences(reviews_list, maxlen=max_length, truncating='pre')

In [80]:
vocab_size = len(tokenizer.word_counts)

In [81]:
print(vocab_size)

54256


In [82]:
# creating the numpy array
reviews_list = np.array(reviews_list)

In [83]:
reviews_list[2]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,    20, 19766,    19,    26,    67,
         173,    97,     1,     9,   290,   299,   396,  1318,   646,
          41,   646,     1,  1979,    29,   205,    10,   214,    85,
          13,    50,   763,   230,  1080,   271,   646], dtype=int32)

In [84]:
# separating the label and the features
X = reviews_list
y = final_df['<outcome>']

In [85]:
len(X)

27910

In [86]:
# convert lable to numerical format as well
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [87]:
# create the numpy array 
y = np.array(y)

In [88]:
# create a two dimensional array 
from keras.utils import to_categorical
y = to_categorical(y)

In [89]:
y

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [94]:
# spliting the dataset into traning, testing and the evaluation set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)




In [95]:
X_train[2]

array([  622,   823, 11490,  1561,  3363,    47,    11,   448,     2,
          60,    32,   125,    99,  1048,  2680, 22875,   286,  1883,
         519,  2398,  2045,    48,  1292,   417,    69,   157, 22876,
        2116,  1886,   510,  1567,   422,  4372,  1672,   956,    27,
          29,  6911,  1194,    89,  2133,  6972,  6833,    32,   125,
         198, 34538,   622,  2785,  1561, 34539,   800,   127,     2,
         182,    60,    37, 14221,   738,    11,  1878, 34540,    65,
        2283,   247,   169,  1561,    48,  2823,   229], dtype=int32)

# Making the Model Ready

In [139]:
# this model refers to the model class we developed which is model.py
# from model import Model

# imports
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, GRU, Bidirectional
from keras.utils.data_utils import pad_sequences
import numpy as np
from keras.preprocessing.text import Tokenizer

from keras.models import load_model


class Model:
    def __init__(self) -> None:
        self.optimizer = tf.keras.optimizers.legacy.Adam
        self.metrices = ["accuracy"]
        self.elipson = None
        self.__model = None
        self.model_name = None
        
    def init_model(self, 
                   seq_len,
                   vocab_size=None, 
                   model_type:'LSTM'="LSTM or GRU or Bidirectional",
                   units=64, 
                   no_of_neurons=8, 
                   learning_rate=0.01,
                   layer=None,
                   droup_out= 0.2,
                   model_name = None
                   ):
        """initialize the neural network model with the given characterstices like model type, units , sequncence-length, vocab-size and learning rate """
        
        # assigning the name
        self.model_name = model_name # for saving the model
        
        # checking and loading the previous model
        try:
            self.__model = self.load_model(self.model_name)
        except OSError:
            print('previous model not found creating new model')
            
            # creating the new model
            self.__model = Sequential()
            self.__model.add(
                Embedding(input_dim=vocab_size, output_dim=30, input_length=seq_len)
            )
            if layer is not None:
                self.__model.add(model_type(layer(units=units)))
            else:
                self.__model.add(model_type(units=units))

            self.__model.add(Dropout(droup_out))
            # self.__model.add(LSTM(units=(units//2)))
            # self.__model.add(Dense(self.neurons, activation=self.activation_func))
            self.__model.add(Dense(2, activation="softmax"))
            self.__model.compile(
                loss="categorical_crossentropy", optimizer=self.optimizer(learning_rate), metrics=self.metrices
            )
        else:
            # checking and loading the model wiights 
            try:
                self.load_model_weights(self.model_name)
            except tf.errors.NotFoundError:
                print('Model weight not found')
        finally:
            # finally compiling the model to match the current hyper parameters
             self.__model.compile(
                loss="categorical_crossentropy", optimizer=self.optimizer(learning_rate), metrics=self.metrices
            )
            
        print(self.__model.summary())



    def init_general_model(self, hidden_layers, no_of_neuraons, activation_func='relu', learning_rate=0.01, model_name='general'):
        """initialize the neural network model with the given characterstices like layer"""
        # initialize the model name
        self.model_name = model_name 
        
         # checking the previous model if exists and also checking for it's weights if not creating new model 
        try:
            self.__model = self.load_model(self.model_name)
        except tf.errors.NotFoundError:
            print('previous model not found creating new model')
            
            # creating the new general model
            self.__model = Sequential()
            self.__model.add(
                Dense(no_of_neuraons // 2, input_dim=70, activation=activation_func))

            for _ in range(hidden_layers):
                self.__model.add(Dense(no_of_neuraons, activation=activation_func))
            # self.__model.add(Dropout(0.2))
            self.__model.add(Dense(2, activation="softmax"))

        else:
            # checking and loading the model wiights 
            try:
                self.load_model_weights(self.model_name)
            except tf.errors.NotFoundError:
                print('Model weight not found')
        finally:
            # compiling the model in any case 
            self.__model.compile(
                loss="categorical_crossentropy",
                optimizer=self.optimizer(learning_rate),
                metrics=self.metrices,)
            
        print(self.__model.summary())


    def train_model(self, X_train, y_train, epochs=100, batch_size=500, verbose=2):
        """train the model using the traning set and default epoch is set to 100, batch_size is 500 and verbose is 2 which can be overridden"""
        self.__model.fit(
            X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose
        )

        # saving the model weights
        self.save_model_weights()

        # save the model
        self.save_model()

        return self.__model

    def evaluate_model(self, X_evaluate, y_evaluate, verbose=2):
        """evaluate the model after the traning have to done to the modfel"""
        self.__model.evaluate(X_evaluate, y_evaluate, verbose=verbose)
        return self.__model

    def test_model(self, X_test):
        """returns the prediction from the model trained"""
        return self.__model.predict(X_test, verbose=2)

    def predict_review(self, review):
        """predict the review is whether postive or negative after traning the model"""
        return self.__model.predict(review, verbose=2)

    def save_model(self ,path="./data/model_weights/"):
        """this will save the entire model to the file"""
        self.__model.save((path + self.model_name))

    def load_model(self, model_name, path="./data/model_weights/"):
        """loads the model that is being saved"""
        return load_model(f'{path}{model_name}')

    def save_model_weights(self, path="./data/model_weights/"):
        """save the model into the file when called"""
        self.__model.save_weights(f'{path}{self.model_name}_weights')

    def load_model_weights(self, model_name, path="./data/model_weights/"):
        """load the model from the the file when called"""
        self.__model.load_weights(f'{path}{model_name}_weights')

    def tokenize_and_pad_items(self, item_seq=None, max_length=70):
        """tokenize the items, converts to the sequence and returns the sequence"""
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(item_seq)
        item_seq = tokenizer.texts_to_sequences(item_seq)
        item_seq = pad_sequences(item_seq, maxlen=max_length, truncating="pre")
        return item_seq


In [157]:
from keras.layers import Dense, LSTM, Embedding, Dropout, GRU, Bidirectional


In [ ]:
# remotes/natural_language_processing/main
# remotes/origin/main

In [109]:
# GRU Model


# initializing model class
model = Model()

# initializing the Model
model.init_model(model_name="GRU",
                 seq_len=max_length,
                 vocab_size=vocab_size,
                 model_type=GRU,
                 units=64,
                 droup_out=0.2,
                 learning_rate=0.001)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 70, 30)            1627680   
                                                                 
 gru_1 (GRU)                 (None, 64)                18432     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,646,242
Trainable params: 1,646,242
Non-trainable params: 0
_________________________________________________________________
None


In [140]:
# LSTM MODEL


# initializing model class
model = Model()

# initializing the Model
model.init_model(model_name="LSTM",
                 seq_len=max_length,
                 vocab_size=vocab_size,
                 model_type=LSTM,
                 units=64,
                 learning_rate=0.01)



previous model not found creating new model
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 70, 30)            1627680   
                                                                 
 lstm_3 (LSTM)               (None, 64)                24320     
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,652,130
Trainable params: 1,652,130
Non-trainable params: 0
_________________________________________________________________
None


In [118]:
# Bi-directional MODEL

# initializing model class
model = Model()

# initializing the Model
model.init_model(model_name='Bidirectional',
                 seq_len=max_length,
                 vocab_size=vocab_size,
                 model_type=Bidirectional,
                 units=32,
                 learning_rate=0.01,
                 droup_out=0.1,
                 layer=LSTM)



previous model not found creating new model
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 70, 30)            1627680   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               16128     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,643,938
Trainable params: 1,643,938
Non-trainable params: 0
_________________________________________________________________
None


In [141]:
# traning the model
model.train_model(X_train, y_train, epochs=20, batch_size=25)

Epoch 1/20
715/715 - 118s - loss: 0.4896 - accuracy: 0.7660 - 118s/epoch - 165ms/step
Epoch 2/20
715/715 - 74s - loss: 0.2261 - accuracy: 0.9123 - 74s/epoch - 104ms/step
Epoch 3/20
715/715 - 79s - loss: 0.0917 - accuracy: 0.9689 - 79s/epoch - 110ms/step
Epoch 4/20
715/715 - 79s - loss: 0.0479 - accuracy: 0.9849 - 79s/epoch - 111ms/step
Epoch 5/20
715/715 - 65s - loss: 0.0300 - accuracy: 0.9895 - 65s/epoch - 91ms/step
Epoch 6/20
715/715 - 79s - loss: 0.0203 - accuracy: 0.9929 - 79s/epoch - 110ms/step
Epoch 7/20
715/715 - 65s - loss: 0.0332 - accuracy: 0.9894 - 65s/epoch - 91ms/step
Epoch 8/20
715/715 - 62s - loss: 0.0347 - accuracy: 0.9891 - 62s/epoch - 87ms/step
Epoch 9/20
715/715 - 110s - loss: 0.0238 - accuracy: 0.9922 - 110s/epoch - 154ms/step
Epoch 10/20
715/715 - 96s - loss: 0.0174 - accuracy: 0.9943 - 96s/epoch - 135ms/step
Epoch 11/20
715/715 - 73s - loss: 0.0133 - accuracy: 0.9960 - 73s/epoch - 103ms/step
Epoch 12/20
715/715 - 75s - loss: 0.0096 - accuracy: 0.9971 - 75s/epoch -

INFO:tensorflow:Assets written to: ./data/model_weights/LSTM/assets


INFO:tensorflow:Assets written to: ./data/model_weights/LSTM/assets


In [142]:
model.evaluate_model(X_val, y_val)

140/140 - 15s - loss: 1.1539 - accuracy: 0.7868 - 15s/epoch - 110ms/step


In [143]:
predictions = np.argmax(model.test_model(X_test), axis=1)
# np.argmax(model.predict(scaled_X_test),axis=1)

175/175 - 3s - 3s/epoch - 18ms/step


In [144]:
predictions

array([1, 1, 1, ..., 0, 1, 0])

In [145]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print(classification_report(predictions, y_test.argmax(axis=1)))
# accuracy_score(predictions, y_test.argmax(axis=1))

              precision    recall  f1-score   support

           0       0.77      0.81      0.79      2682
           1       0.81      0.77      0.79      2900

    accuracy                           0.79      5582
   macro avg       0.79      0.79      0.79      5582
weighted avg       0.79      0.79      0.79      5582



In [155]:
# let's predict a review 
r_p = "never gonna buy this product it was horrible and cannot operate a single time "
# r__p = [r_p.split(" ")]
r_p = nlp(r_p)
r_p = clean_review([r_p], nlp)
print(r_p[0])
r_p = tokenize_items(tokenizer, r_p[0])
r_p = pad_sequences(r_p, maxlen=max_length, truncating='pre')


print(r_p)
n_g = "The product was good the quality was sturdy however it was not long lasting. I will purchase again though. "
# n_g = [n_g.split(" ")]
n_g = nlp(n_g)
n_g = clean_review([n_g], nlp)
print(n_g[0])
n_g = tokenize_items(tokenizer, n_g[0])
n_g = pad_sequences(n_g, maxlen=max_length, truncating='pre')

print(r_p)
print(np.argmax(model.predict_review(n_g), axis=1))

# print(np.argmax(model.predict_review(n_g), axis=1))
# print()

[['not', 'like', 'product', 'look', 'cheap', 'not', 'job', '\xa0']]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    1    7   26   19  188    1  184 4676]]
[['the', 'product', 'good', 'quality', 'sturdy', 'not', 'long', 'last', 'purchase', '\xa0']]
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    1    7   26   19  188    1  184 4676]]
1/1 - 0s - 14ms/epoch - 14ms/step
[1]


In [ ]:
# removing stopping words
# i = 0
# list_ref = []
# for tokens in tokens_list:
#     tokens_ = [str(token) for token in tokens if token.text not in nlp.Defaults.stop_words]
#     if len(tokens) != len(tokens_):
#         # print(f"original ---> {len(tokens)} changed --> {len(tokens_)}")
#         i += 1
#     list_ref.append(tokens_)
# print(i)
    
    
# def make_spacy_doc(token_list):
#     return nlp.make_doc(' '.join(token_list))


# tokens_list_ref = []
# for tokens in list_ref:
#     tokens_list_ref.append(make_spacy_doc)

# for sentence in tokens_list_ref[0].sents:
#     print(sentence)


# for word in reviews[6].split(' '):
#         if word in nlp.Defaults.stop_words:
#             print(word)

7959


In [ ]:
# # truncate the ':electronics' from the product name column values
# df_positive['<product_name>'] = df_positive['<product_name>'].str.partition(":")[0]

# #working with the unique id and the reviewer
# df_positive['<unique_id>'][98]

# col = final_df.columns.to_list()
# col = col[:-1]
# print(col)

In [ ]:
# # split the data
# train_data,test_data,validate_data = final_df.randomSplit([0.7, 0.15, 0.15], seed=50)
# train_data.persist()
# test_data.persist()
# validate_data.persist()
# print("Number of records for training data : {}".format(train_data.count()) + " rows")
# print("Number of records for test data : {}".format(test_data.count()) + " rows")
# print("Number of records for test data : {}".format(validate_data.count()) + " rows")

In [ ]:
# filtered_reviews = []
# for review_text in reviews_text:
#     sentence = pick_words(review_text, 138)
#     filtered_reviews.append(sentence)
# print(filtered_reviews)
# #convert to dataframe
# filtered_reviews = pd.DataFrame(filtered_reviews, columns=["<review_text"])
# df_positive['<review_text>'] =  filtered_reviews

# text = df_positive['<review_text>']

# def words_count(sentence):
#     """
#     This function takes a sentence and a desired number of words as input
#     and returns a string containing only the specified number of words.
#     """
#     words = sentence.split()
#     # print(len(words))
#     return len(words)


# for t in text:
#     if words_count(t) > 138:
#         print('greater')

# df_positive.columns

# df_positive['<outcome>'] = 'positive'
# df_negative['<outcome>'] = 'negative'

# review_text
# ratings
# title
# helpful

# columns_to_include = ["<review_text>", "<rating>", "<title>"]
# df_positive_new = df_positive[['<review_text>', '<rating>', '<title>', '<outcome>']]

# # data_frame for the the postivie reviews
# df_positive_new.isna().sum()

# data frame for the negative reviews
# neg_reviews = df_negative['<review_text>']
# fil_neg_review = []

# for text in neg_reviews:
#     sentence = pick_words(text, 138)
#     fil_neg_review.append(sentence)
    
# df_negative['<review_text>'] = pd.DataFrame(filtered_reviews, columns=["<review_text"])


# rev  = df_negative['<review_text>']
# for r in rev:
#     print(words_count(r))


# df_negative_new = df_negative[['<review_text>', '<rating>', '<title>', '<outcome>']]

# # adding the table outcome to the both dataframe
# outcome = []
# for i in range(len(df_positive_new)):
#     outcome.append('positive')



['I purchased this unit due to frequent blackouts in my area and 2 power supplies going bad. It will run my cable modem, router, PC, and LCD monitor for 5 minutes. This is more than enough time to save work and shut down. Equally important, I know that my electronics are receiving clean power.I feel that this investment is minor compared to the loss of valuable data or the failure of equipment due to a power spike or an irregular power supply.As always, Amazon had it to me in &lt;2 business days', "I ordered 3 APC Back-UPS ES 500s on the recommendation of an employee of mine who used to work at APC. I've had them for about a month now without any problems. They've functioned properly through a few unexpected power interruptions. I'll gladly order more if the need arises.Pros: - Large plug spacing, good for power adapters - Simple design - Long cordCons: - No line conditioning (usually an expensive option", "Wish the unit had a separate online/offline light. When power to the unit is mi